In [1]:
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import langchain
import pickle
import time

In [ ]:
import os
# Set OpenAI API Key
os.environ["OPENAI_API_KEY"] = ""
os.environ["LLAMA_API_KEY"]= ""

In [ ]:
llm = OpenAI(temperature = 0.3, max_tokens=500)

C:\Users\Harish\AppData\Local\Temp\ipykernel_15104\203484346.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(temperature = 0.3, max_tokens=500)


In [4]:
from langchain.document_loaders import CSVLoader

# Path to your CSV file
file_path = "D:\LLMs\FProject_2\data\insurance.csv" 

# Load the CSV using CSVLoader
loader = CSVLoader(file_path=file_path,encoding="utf-8")



<>:4: SyntaxWarning: invalid escape sequence '\L'
<>:4: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Harish\AppData\Local\Temp\ipykernel_15104\2826097029.py:4: SyntaxWarning: invalid escape sequence '\L'
  file_path = "D:\LLMs\FProject_2\data\insurance.csv"


In [5]:
data = loader.load()

In [6]:
len(data)

107

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 100
)

docs = text_splitter.split_documents(data)

In [8]:
len(docs)

161

In [9]:

# Initialize the OpenAI embeddings model
embeddings = OpenAIEmbeddings()


C:\Users\Harish\AppData\Local\Temp\ipykernel_15104\1681419414.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()


In [13]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
  Using cached faiss_cpu-1.9.0.post1-cp312-cp312-win_amd64.whl.metadata (4.5 kB)
Using cached faiss_cpu-1.9.0.post1-cp312-cp312-win_amd64.whl (13.8 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# Create a FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)

In [15]:
# Save the vectorstore to disk
faiss_path = "faiss_store_insurance_rag"
vectorstore.save_local(faiss_path)

In [16]:
# Optional: Save the metadata with pickle
with open(f"{faiss_path}/docstore.pkl", "wb") as f:
    pickle.dump(vectorstore.docstore, f)

print(f"FAISS vector database saved to {faiss_path}")

FAISS vector database saved to faiss_store_insurance_rag


In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm (
    llm = llm,
    retriever = vectorstore.as_retriever()
)

In [19]:
query= "What is the average life insurance cost per month?"

In [20]:
langchain.debug = True
chain({"question" :  query}, return_only_outputs = True)

C:\Users\Harish\AppData\Local\Temp\ipykernel_15104\500905448.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  chain({"question" :  query}, return_only_outputs = True)


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is the average life insurance cost per month?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "category: Insurance\nsubCategory: Life Insurance\nquestion: What is the average life insurance cost per month?\nanswer: The cost of a life insurance policy depends on the type of policy you own. Term plans are generally cheaper in nature as it only offers death benefit with no profits or returns. Traditional plans and unit-linked plans tend to cost more as they offer a wide range of benefits. The cost also depends on the sum assured i.e. a higher sum assured will cost you more and vice versa.",
      "question": "What is the average life insuranc

{'answer': " The average cost of a life insurance plan can vary depending on a variety of factors, such as the policyholder's age, health, and coverage amount. On average, a life insurance plan can cost anywhere from $50 to $200 per month. However, this cost can also be affected by the type of policy and any additional features or riders included. It is important to carefully consider all of these factors when determining the cost of a life insurance plan.\n",
 'sources': 'D:\\LLMs\\FProject_2\\data\\insurance.csv'}